In the following code I executed SQL queries to retrieve data from the database, including finding the top 5 most popular stories by score, determining if the website is dominated by a small percentage of users, finding the number of times a specific video link was posted by each user, and categorizing the source of each story using a CASE statement. 

The advanced SQL skills used in this code include grouping data with the GROUP BY clause, filtering data with the HAVING clause, and using aggregate functions like SUM and COUNT.

## Setup

In [1]:
import pandas as pd 

In [2]:
import sqlite3

In [3]:
df = pd.read_csv("hacker_news.csv")

In [4]:
cnn = sqlite3.connect('jupyter_sql.db')

In [5]:
df.to_sql('hacker_news',cnn, if_exists='replace')

In [6]:
%load_ext sql

In [7]:
%sql sqlite:///jupyter_sql.db

In [22]:
%%sql 

select * from hacker_news
limit 5;

 * sqlite:///jupyter_sql.db
Done.


index,title,user,score,timestamp,url
0,None,coldtea,None,2014-01-27T17:31:13Z,None
1,None,etanol,None,2011-10-23T18:46:40Z,None
2,None,None,None,2016-02-28T06:26:56Z,None
3,None,Locke1689,None,2014-08-12T22:13:10Z,None
4,None,miloshadzic,None,2013-03-06T12:28:02Z,None


### information about the data:

title: the title of the story

user: the user who submitted the story

score: the score of the story

timestamp: the time of the story

url: the link of the story


# Starting the code

### Start by getting a feel for the hacker_news table! Let’s find the most popular Hacker News stories:

In [9]:
%%sql
select title, score 
from hacker_news
order by 2 desc
limit 5;

 * sqlite:///jupyter_sql.db
Done.


title,score
Penny Arcade � Surface Pro 3 update,517.0
Hacking The Status Game,309.0
Postgres CLI with autocompletion and syntax highlighting,304.0
Stephen Fry hits out at �infantile� culture of trigger words and safe spaces,282.0
Reversal: Australian Govt picks ODF doc standard over Microsoft,191.0


## Hacker News Moderating


Recent studies have found that online forums tend to be dominated by a small percentage of their users (1-9-90 Rule).

Is this true of Hacker News?

Is a small percentage of Hacker News submitters taking the majority of the points?

### First, find the total score of all the stories.

In [10]:
%%sql
select sum(score) as 'total_score'
from hacker_news;


 * sqlite:///jupyter_sql.db
Done.


total_score
6366.0


Next, we need to pinpoint the users who have accumulated a lot of points across their stories.

Find the individual users who have gotten combined scores of more than 200, and their combined scores.

GROUP BY and HAVING are needed!

In [11]:
%%sql
select user, sum(score) as "total user score"
from hacker_news
group by user
having sum(score) > 200
order by sum(score) desc;

 * sqlite:///jupyter_sql.db
Done.


user,total user score
vxNsr,517.0
amirkhella,309.0
dmmalam,304.0
metafunctor,282.0


Then, we want to add these users’ scores together and divide by the total to get the percentage.

Add their scores together and divide it by the total sum. Like so:

SELECT (1.0 + 2.0 + 3.0) / 6.0;


So, is Hacker News dominated by these users?

In [12]:
%%sql
SELECT (517 + 309 + 304 + 282) / 6366.0 as "percent of score by 4 users";

 * sqlite:///jupyter_sql.db
Done.


percent of score by 4 users
0.2218033301916431


It is seems like the website is dominated by this users

Oh no! While we are looking at the power users, some users are rickrolling — tricking readers into clicking on a link to a funny video and claiming that it links to information about coding.

The url of the video is:

https://www.youtube.com/watch?v=dQw4w9WgXcQ

How many times has each offending user posted this link?

In [13]:
%%sql
select user, count(url)
from hacker_news
where url like '%watch?v=dQw4w9WgXcQ'
group by user;

 * sqlite:///jupyter_sql.db
Done.


user,count(url)
scorpiosister,1
sonnynomnom,1


## Which sites feed Hacker News?

Hacker News stories are essentially links that take users to other websites.

Which of these sites feed Hacker News the most:

GitHub, Medium, or New York Times?




First, we want to categorize each story based on their source.

We can do this using a CASE statement:

In [14]:
%%sql
select distinct url
from hacker_news
where url like '%times%'
LIMIT 3;

 * sqlite:///jupyter_sql.db
Done.


url
http://www.nytimes.com/2016/08/26/business/dealbook/cycling-matches-the-pace-and-pitches-of-tech.html
http://www.nytimes.com/2011/03/06/books/review/Kois-t.html?_r=1&ref=review&pagewanted=all
http://www.latimes.com/business/la-fi-ct-piracy-2010070102871905.story


In [23]:
%%sql
select case 
    when url like "%github.com%" then 'GitHub'
    when url like "%Medium.com%" then 'Medium'
    when url like "%nytimes.com%" then 'New York Times'
    ELSE 'Other'
    END AS 'Source'
from hacker_news
limit 5;

 * sqlite:///jupyter_sql.db
Done.


Source
Other
Other
Other
Other
Other


 Next, build on the previous query:

Add a column for the number of stories from each URL using COUNT().

Also, GROUP BY the CASE statement.

Remember that you can refer to a column in GROUP BY using a number.

In [16]:
%%sql
select case 
    when url like "%github.com%" then 'GitHub'
    when url like "%Medium.com%" then 'Medium'
    when url like "%nytimes.com%" then 'New York Times'
    ELSE 'Other'
  END AS 'Source',
  count(*)
from hacker_news
GROUP BY 1;

 * sqlite:///jupyter_sql.db
Done.


Source,count(*)
GitHub,23
Medium,12
New York Times,13
Other,3952


Every submitter wants their story to get a high score so that the story makes it to the front page, but…

What’s the best time of the day to post a story on Hacker News?

Before we get started, let’s run this query and take a look at the timestamp column:

In [17]:
%%sql
SELECT timestamp
FROM hacker_news
LIMIT 10;


 * sqlite:///jupyter_sql.db
Done.


timestamp
2014-01-27T17:31:13Z
2011-10-23T18:46:40Z
2016-02-28T06:26:56Z
2014-08-12T22:13:10Z
2013-03-06T12:28:02Z
2011-04-16T21:04:23Z
2014-03-18T21:44:46Z
2012-11-19T11:54:38Z
2016-11-04T13:55:30Z
2016-07-02T22:54:47Z


Notice that the values are formatted like:

2018-05-08T12:30:00Z

If you ignore the T and Z, the format is:

YYYY-MM-DD HH:MM:SS


SQLite comes with a strftime() function - a very powerful function that allows you to return a formatted date.

It takes two arguments:

strftime(format, column)

Let’s test this function out:

In [24]:
%%sql
SELECT timestamp,
   strftime('%H', timestamp)
FROM hacker_news
GROUP BY 1
LIMIT 5;

 * sqlite:///jupyter_sql.db
Done.


timestamp,"strftime('%H', timestamp)"
None,None
2007-03-16T20:52:19Z,20
2007-04-03T03:04:09Z,03
2007-05-01T03:11:17Z,03
2007-05-05T05:43:58Z,05


This returns the hour, HH, of the timestamp column!

Okay, now we understand how strftime() works. Let’s write a query that returns three columns:

The hours of the timestamp
The average score for each hour
The count of stories for each hour

In [25]:
%%sql
SELECT strftime('%H', timestamp) as 'hour of the day', avg(score), count(*)
from hacker_news
group by 1
order by 2 desc
limit 10;

 * sqlite:///jupyter_sql.db
Done.


hour of the day,avg(score),count(*)
18,27.02777777777778,266
07,21.333333333333332,104
19,20.413793103448278,238
20,18.275862068965516,239
12,14.310344827586206,123
09,11.125,119
16,9.755555555555556,237
15,9.491228070175438,268
17,8.86046511627907,215
23,8.434782608695652,170



Let’s edit a few things in the previous query:

Round the average scores (ROUND()).

Rename the columns to make it more readable (AS).

Add a WHERE clause to filter out the NULL values in timestamp.

Take a look at the result again:

What are the best hours to post a story on Hacker News?

In [26]:
%%sql
SELECT strftime('%H', timestamp) as 'hour of the day', round(avg(score)) as "average score", count(*)as "number of stories"
from hacker_news
where strftime('%H', timestamp) is not null 
group by 1
order by 2 desc
limit 10;

 * sqlite:///jupyter_sql.db
Done.


hour of the day,average score,number of stories
18,27.0,266
07,21.0,104
19,20.0,238
20,18.0,239
12,14.0,123
09,11.0,119
16,10.0,237
17,9.0,215
15,9.0,268
23,8.0,170


The best hours to post a story is either 18:00 or 7:00